# BPM Data Import from DDF

### General Settings

In [8]:
import pandas as pd
import numpy as np
#import re as re
#import itertools as it
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from pandas.io import sql
#from shutil import copyfile
#from subprocess import Popen
import winsound

In [9]:
#pd.set_option('display.max_columns', 1000)
#pd.set_option('display.max_rows', 10)

### Set parameter

In [10]:
p_countrycode='THA'
p_mode='a' # r=replace, a = append

### Set directories

In [11]:
p_dir=r'\\TSHAMFIL901\iClick\Projects\Allianz\DB\Chartpack\XLSX'.replace(chr(92),'/')

In [12]:
p_subdir=p_dir +'/'+ r'data'.replace(chr(92),'/')
p_xlsx = 'FPI'

In [13]:
engine_dv_bpm = create_engine('mssql+pyodbc://TSMMHSQVS901/DV_BPM?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes')

In [14]:
df=pd.read_sql_query(
    """SELECT *
    FROM DV_BPM.dbo.xls_FPI
    """, engine_dv_bpm)

In [15]:
dfg=df.groupby(['Country','LoBID','Year'],as_index=False)['SQR4'].mean()

In [16]:
df=pd.merge(left=df, right=dfg, left_on=('Country','LoBID','Year'), right_on=('Country','LoBID','Year'), how='left', suffixes=['','_mean'])

In [17]:
df['FPI']=(df.SQR4/df.SQR4_mean)*100

In [18]:
df_delta=df[['Country','LoBID','Brand','Year','FPI']].copy()

In [19]:
df_delta['Year']=df_delta.Year+1

In [20]:
df=pd.merge(left=df, right=df_delta, left_on=('Country','LoBID','Brand','Year'), right_on=('Country','LoBID','Brand','Year'), how='left', suffixes=['','_PY'])

In [21]:
df['FPI_delta']=df.FPI-df.FPI_PY

In [22]:
#df[df.Country!='Ireland'].sort_values(['Country','LoBID','Brand','Year'], ascending=[1,1,1,0])

In [ ]:
#session = sessionmaker(bind=engine_dv_bpm)()
#session.execute('sp_update_Touchpoint_UID_level')

In [23]:
for country in df.Country.unique():
    p_country=country
    writer = pd.ExcelWriter(p_dir+'/'+p_xlsx+'_'+p_country+'.xlsx', engine='xlsxwriter')
    workbook  = writer.book

    df[df.Country==p_country].sort_values(['Country','LoBID','Brand','Year'], ascending=[1,1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startcol=1, startrow=4)
    worksheet = writer.sheets['FPI']
    writer.save()

In [ ]:
import winsound
Freq = 3000 # Set Frequency To 3000 Hertz
Dur = 1000 # Set Duration To 1000 ms == 1 second
#winsound.Beep(Freq,Dur)